### **Create CNN Model and Optimize it using Keras Tuner**

In [12]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=21c2ea5e786f68138e2314af5028a3a78a557e76d8ef8cc99272e2a6fe330f04
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=330bdc5f8e2b62c5e6c961155ff44a8eb62b03805bfb270ba45f3ef863ad4b5a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
print(tf.__version__)

2.4.0


In [15]:
fashion_mnist=keras.datasets.fashion_mnist

In [16]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [17]:
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [20]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 16s]
val_accuracy: 0.9111666679382324

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 01m 34s
INFO:tensorflow:Oracle triggered exit


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               4128880   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 4,186,330
Trainable params: 4,186,330
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1046 - accuracy: 0.9619 - val_loss: 0.2644 - val_accuracy: 0.9135
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0664 - accuracy: 0.9758 - val_loss: 0.3295 - val_accuracy: 0.9170
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0439 - accuracy: 0.9840 - val_loss: 0.3561 - val_accuracy: 0.9092
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0302 - accuracy: 0.9892 - val_loss: 0.4042 - val_accuracy: 0.9148
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0204 - accuracy: 0.9926 - val_loss: 0.4724 - val_accuracy: 0.9123
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0181 - accuracy: 0.9941 - val_loss: 0.5281 - val_accuracy: 0.9070
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0177 - accuracy: 0.9941 - val_loss: 0.5361 - val_accuracy